# Program

In [ ]:
with open('book_analyzer.in', 'r') as txt:
    data = txt.readlines()
    txt.close()

def BookAnalyzer(target_size):
    
    #Initialize hashmaps
    market_log = {}
    prices = {'B':[], 'S':[]}

    for line in data:

        #Convert string to list of strings and store variables
        log = line.split()
        order_id = log[2]
        size = int(log[-1])
        size_check = target_size

        #Populate hashmaps if 'A'
        if log[1] == 'A':
            side = log[3]
            price = float(log[4])
            market_log[order_id] = [side] + [price] + [size]
            prices[side].append(price)
            orders = [order[-1] for order in market_log.values() if order[0] == side]

        #Depopulate hashmaps if 'R'
        else:
            side = market_log[order_id][0]
            market_log[order_id][-1] -= size
            orders = [order[-1] for order in market_log.values() if order[0] == market_log[order_id][0]]

            #Remove log if amount hits 0
            if market_log[order_id][-1] == 0:
                prices[market_log[order_id][0]].remove(market_log[order_id][1])
                del market_log[order_id]                

        #No output until orders reach target size
        if sum(orders) < size_check:
            pass

        #Output if orders reach target size
        else:
            total = 0
            orders_used = []

            #Buy output
            if side == 'S':
                while size_check > 0:

                    #Isolate asks
                    sell = {k:v for k,v in market_log.items() if ((k not in orders_used) & (side in v))}
                    sell = {k:v for k,v in sell.items() if (v[-2]==min(v[1] for v in list(sell.values())))}
                    sell = {min(sell, key=sell.get): sell[min(sell, key=sell.get)]}
                    orders_used.append(list(sell.keys())[0])
                    total += sum([v[-2]*min(v[-1],size_check) for v in sell.values()])
                    size_check -= sum([v[-1] for v in sell.values()])
                print('{} {} {}'.format(log[0], 'B', total))

            #Sell output
            else:
                while size_check > 0:
                    buy = {k:v for k,v in market_log.items() if ((k not in orders_used) & (side in v))}
                    buy = {k:v for k,v in buy.items() if (v[-2]==max(v[1] for v in list(buy.values())))}
                    buy = {max(buy, key=buy.get): buy[max(buy, key=buy.get)]}
                    orders_used.append(list(buy.keys())[0])
                    total += sum([v[-2]*min(v[-1],size_check) for v in buy.values()])
                    size_check -= sum([v[-1] for v in buy.values()])
                print('{} {} {}'.format(log[0], 'S', total))

# How did you choose your implementation language?

I chose Python because it is the software language I feel most confident in. I also don't think this could be possible in SQL (maybe R), but I could be wrong.

# How did you arrive at your final implementation? Were there other approaches that you considered or tried first?

From the beginning, I knew that I would have to utilize dictionaries. However, I did not store my variables initially, which left the code unreadable. Even after storing the variables, I still think improvements could be made to readability.

# How does your implementation scale with respect to the target size?

I'm confident that my implementation scales well with target size. I do not believe target size will be the limiting factor (except for unreasonably large numbers)

# How does your implementation scale with respect to the number of orders in the book?

The number of orders in a book would be the limiting factor with my implementation. Using Python, I was able to read in the entire log file (of which there are more than 1M entires), but the output began to slow down my computer before I interrupted the program. Therefore, it is not exactly the number of orders in the book that directly affects the program, but the extreme number of lines in the resulting output.

# Additional Notes

There are a few key differences between my output and the sample output.

> 1. My output did not include any "NA" outputs. Instead, if the order size did not meet the target size requirement, my program simply skipped the row and moved on to the next.
> 2. My output would not skip rows in which multiple logs occurred within the same milliseconds. As a result, if timestamp "x" had 2 actions occur at the same time, my output would have a separate, duplicate row for each timestamp row.